## 보스턴 주택 가격 데이터셋

1970년 중반 보스턴 외곽 지역의 범죄율, 지방세율 등의 데이터가 주어졌을 때 주택 가격의 중간 값을 예측해 보겠습니다.

여기서 사용할 데이터셋은 이전 두 개의 예제와 다릅니다. 데이터 포인트가 506개로 비교적 개수가 적고 404개는 훈련 샘플로 102개는 테스트 샘플로 나누어져 있습니다. 입력 데이터에 있는 각 특성(예를 들어 범죄율)은 스케일이 서로 다릅니다. 어떤 값은 0과 1 사이의 비율을 나타내고 어떤 것은 1과 12 사이의 값을 가지거나 1과 100 사이의 값을 가집니다.

In [1]:
from tensorflow.keras.datasets import boston_housing

(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [2]:
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

In [4]:
# 데이터 준비
# 테스트 데이터를 정규화할 때 사용한 값이 훈련 데이터에서 계산한 값임을 주목하세요. 
# 머신 러닝 작업 과정에서 절대로 테스트 데이터에서 계산한 어떤 값도 사용해서는 안 됩니다. 
# 데이터 정규화처럼 간단한 작업조차도 그렇습니다.40
mean = train_data.mean(axis=0)
train_data -= mean
std=train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [5]:
print(train_data.mean())
print(train_data.std())

2.602301889250862e-15
0.9999999999999994


In [6]:
train_data[:5]

array([[-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
        -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
         1.14850044,  0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, -0.25683275, -1.21518188,
         1.89434613, -1.91036058,  1.24758524, -0.85646254, -0.34843254,
        -1.71818909,  0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , -0.25683275,  0.62864202,
        -1.82968811,  1.11048828, -1.18743907,  1.67588577,  1.5652875 ,
         0.78447637,  0.22061726, -1.30850006],
       [-0.40149354, -0.48361547, -0.86940196, -0.25683275, -0.3615597 ,
        -0.3245576 , -1.23667187,  1.10717989, -0.51114231, -1.094663  ,
         0.78447637,  0.44807713, -0.65292624],
       [-0.0056343 , -0.48361547,  1.0283258 , -0.25683275,  1.32861221,
         0.15364225,  0.69480801, -0.57857203,  1.67588577,  1.5652875 ,
         0.78447637,  0.3898823 ,  0.26349695]])

In [7]:
# 모델 구성
from tensorflow.keras import models, layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64,activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

In [8]:
len(train_data)//4

101

In [9]:
import numpy as np
k = 4
num_val_samples = len(train_data)//k
num_epochs=100
all_score=[]
for i in range(k):
  val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
  val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
  partial_train_data = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
  partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
  # 케라스 모델 구성
  model = build_model()
  model.fit(partial_train_data,partial_train_targets,
            epochs=num_epochs, batch_size=1,verbose=1)
  val_mse, val_mae = model.evaluate(val_data,val_targets,verbose=1)
  all_score.append(val_mae)


Epoch 1/100
303/303 [==============================] - 1s 2ms/step - loss: 218.3251 - mae: 11.0131
Epoch 2/100
303/303 [==============================] - 1s 2ms/step - loss: 25.9379 - mae: 3.5003
Epoch 3/100
303/303 [==============================] - 1s 2ms/step - loss: 19.1528 - mae: 2.9061
Epoch 4/100
303/303 [==============================] - 1s 2ms/step - loss: 17.7234 - mae: 2.8058
Epoch 5/100
303/303 [==============================] - 0s 2ms/step - loss: 15.2505 - mae: 2.6037
Epoch 6/100
303/303 [==============================] - 0s 2ms/step - loss: 14.0993 - mae: 2.5157
Epoch 7/100
303/303 [==============================] - 0s 2ms/step - loss: 13.7494 - mae: 2.4395
Epoch 8/100
303/303 [==============================] - 0s 2ms/step - loss: 13.0096 - mae: 2.3911
Epoch 9/100
303/303 [==============================] - 1s 2ms/step - loss: 12.2858 - mae: 2.3667
Epoch 10/100
303/303 [==============================] - 0s 2ms/step - loss: 12.4441 - mae: 2.3591
Epoch 11/100
303/303 [=====

In [10]:
np.mean(all_score)

2.383667826652527

In [12]:
from tensorflow.keras import backend as K
#메모리 해제
K.clear_session()

num_epochs=500  적용하여 모델 학습 및  평가를 수행ㄹ

In [13]:
import numpy as np
k = 4
num_val_samples = len(train_data)//k
num_epochs=500
all_score=[]
for i in range(k):
  val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
  val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
  partial_train_data = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
  partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
  # 케라스 모델 구성
  model = build_model()
  model.fit(partial_train_data,partial_train_targets,
            epochs=num_epochs, batch_size=1,verbose=1)
  val_mse, val_mae = model.evaluate(val_data,val_targets,verbose=1)
  all_score.append(val_mae)


Epoch 1/500
303/303 [==============================] - 1s 2ms/step - loss: 202.9847 - mae: 10.7200
Epoch 2/500
303/303 [==============================] - 1s 2ms/step - loss: 28.5643 - mae: 3.6016
Epoch 3/500
303/303 [==============================] - 1s 2ms/step - loss: 21.2861 - mae: 3.0414
Epoch 4/500
303/303 [==============================] - 1s 2ms/step - loss: 18.3369 - mae: 2.7905
Epoch 5/500
303/303 [==============================] - 1s 2ms/step - loss: 16.4302 - mae: 2.6231
Epoch 6/500
303/303 [==============================] - 1s 2ms/step - loss: 15.2836 - mae: 2.4960
Epoch 7/500
303/303 [==============================] - 1s 2ms/step - loss: 14.5813 - mae: 2.4336
Epoch 8/500
303/303 [==============================] - 1s 2ms/step - loss: 13.5591 - mae: 2.3904
Epoch 9/500
303/303 [==============================] - 0s 2ms/step - loss: 13.1239 - mae: 2.3560
Epoch 10/500
303/303 [==============================] - 1s 2ms/step - loss: 12.3132 - mae: 2.2966
Epoch 11/500
303/303 [=====

In [14]:
np.mean(all_score)

2.749394476413727